**Pomembni stolpci**

In [1]:
import pandas as pd

df = pd.read_csv('sample.csv', sep=',')

pomembni = df[['smart_5_raw','smart_187_raw','smart_188_raw']]
display(pomembni)
display(pomembni.isnull().sum())

# !!odpovedi je samo 8, rabimo utezi, vec takih primerov iz drugih datasetov...
odpovedi = df.loc[df['failure'] == 1]
display(odpovedi)

,smart_5_raw,smart_187_raw,smart_188_raw
0,0.0,0.0,NaN
1,0.0,0.0,NaN
2,0.0,0.0,NaN
3,0.0,0.0,NaN
4,0.0,0.0,NaN
...,...,...,...
329482,0.0,NaN,NaN
329483,0.0,NaN,NaN
329484,0.0,NaN,NaN
329485,0.0,NaN,NaN


smart_5_raw        2264
smart_187_raw    216540
smart_188_raw    217083
dtype: int64

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
24793,2025-09-29,5PHJS4PD,HGST HUH721212ALE604,12000138625024,1,phx1,0,1059,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24823,2025-09-29,5PHKJGKF,HGST HUH721212ALE604,12000138625024,1,phx1,0,1059,4,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98004,2025-09-29,AAG6XXUH,HGST HUH721212ALN604,12000138625024,1,sac0,0,1120,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102179,2025-09-29,2AGMHTDY,HGST HUH721212ALN604,12000138625024,1,sac2,0,1124,16,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102249,2025-09-29,2AGMNJKY,HGST HUH721212ALN604,12000138625024,1,sac2,0,1124,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241207,2025-09-29,7190A5JZFVGG,TOSHIBA MG08ACA16TE,16000900661248,1,phx1,40,2001,19,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288562,2025-09-29,5BGAZ5SL,WDC WUH721816ALE6L4,16000900661248,1,iad1,50,1009,8,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311063,2025-09-29,ZYD1R3YM,ST24000NM002H,24000277250048,1,iad1,50,1028,2,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Rough Preprocessing**

odstranjevanje vecine NaN vrednosti, fokus na pomembne stolpce, ....

In [2]:
import pandas as pd

df = pd.read_csv('sample.csv', sep=',')
display(df.head(5))
display(df.shape)

NaN = df.isnull().mean()

nujni_stolpci = [
    'smart_5_raw', 
    'smart_187_raw', 
    'smart_188_raw', 
    'smart_197_raw', 
    'smart_198_raw', 
    'smart_9_raw', 
    'smart_194_raw', 
    'capacity_bytes', 
    'model',
    'failure'
]

#obdrži stolpec, če ima < 30% NaN ALI če je na seznamu nujnih
relevantni = NaN[(NaN < 0.3) | (NaN.index.isin(nujni_stolpci))].index.tolist()

print(f"Obdržali bomo {len(relevantni)} stolpcev.")

zaStran = [col for col in df.columns if col not in relevantni]
df = df.drop(columns=zaStran)

display(df.shape)
display(df)

# "lokacijski stolpci" od blackblaze fizicnih serverjev, ki jih ne rabimo
lokacijski = df[['datacenter','cluster_id','vault_id','pod_id','pod_slot_num','is_legacy_format']]

#ostali nepomembni
nepomembniOstalo = df[['date','serial_number']]

# vsi "normalized" so samo interpetacija proizvajalca, ker je vec proizvajalcev so nekonsistentni
def vsiNorm(df):
    normStolpci = []

    for col in df.columns:
        if 'normalized' in col:
            normStolpci.append(col)
    return normStolpci

normStolpci = vsiNorm(df)
df = df.drop(columns=normStolpci)
df = df.drop(columns=lokacijski)
df = df.drop(columns=nepomembniOstalo)
display(df.shape)   #19 dejansko uporabnih stolpcev ..
display(df.columns)

#df.to_csv('izbolsani_podatki.csv', index=False)

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2025-09-29,2206E608DB42,CT250MX500SSD1,250059350016,0,sac0,0,1028,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-09-29,2207E60CC65A,CT250MX500SSD1,250059350016,0,sac0,0,1028,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-09-29,2340E87B92B5,CT250MX500SSD1,250059350016,0,sac0,0,1028,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-09-29,2340E87B97E8,CT250MX500SSD1,250059350016,0,sac0,0,1028,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-09-29,2407E896B6D5,CT250MX500SSD1,250059350016,0,sac0,0,1028,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(329487, 197)

Obdržali bomo 41 stolpcev.


(329487, 41)

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw
0,2025-09-29,2206E608DB42,CT250MX500SSD1,250059350016,0,sac0,0,1028,4,NaN,...,NaN,NaN,70.0,30.0,100.0,0.0,100.0,0.0,100.0,0.0
1,2025-09-29,2207E60CC65A,CT250MX500SSD1,250059350016,0,sac0,0,1028,13,NaN,...,NaN,NaN,65.0,35.0,100.0,0.0,100.0,0.0,100.0,0.0
2,2025-09-29,2340E87B92B5,CT250MX500SSD1,250059350016,0,sac0,0,1028,14,NaN,...,NaN,NaN,67.0,33.0,100.0,0.0,100.0,0.0,100.0,0.0
3,2025-09-29,2340E87B97E8,CT250MX500SSD1,250059350016,0,sac0,0,1028,2,NaN,...,NaN,NaN,64.0,36.0,100.0,0.0,100.0,0.0,100.0,0.0
4,2025-09-29,2407E896B6D5,CT250MX500SSD1,250059350016,0,sac0,0,1028,8,NaN,...,NaN,NaN,72.0,28.0,100.0,0.0,100.0,0.0,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329482,2025-09-29,3ZHG6U1Z,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,4,48.0,...,100.0,328.0,64.0,32.0,100.0,0.0,100.0,0.0,100.0,0.0
329483,2025-09-29,3ZHG6U9Z,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,2,24.0,...,100.0,327.0,71.0,26.0,100.0,0.0,100.0,0.0,100.0,0.0
329484,2025-09-29,ZGG6LV5B,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,11,8.0,...,100.0,327.0,64.0,32.0,100.0,0.0,100.0,0.0,100.0,0.0
329485,2025-09-29,ZGG7J1TA,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,7,0.0,...,100.0,330.0,67.0,29.0,100.0,0.0,100.0,0.0,100.0,0.0


(329487, 19)

Index(['model', 'capacity_bytes', 'failure', 'smart_1_raw', 'smart_3_raw',
       'smart_4_raw', 'smart_5_raw', 'smart_7_raw', 'smart_9_raw',
       'smart_10_raw', 'smart_12_raw', 'smart_187_raw', 'smart_188_raw',
       'smart_192_raw', 'smart_193_raw', 'smart_194_raw', 'smart_197_raw',
       'smart_198_raw', 'smart_199_raw'],
      dtype='object')

**Vrte diskov**

In [3]:
unikatneVrsteDiskov = df['model'].unique()
print("vrste diskov: ", unikatneVrsteDiskov)

ssd_keywords = ['SSD', 'MTFD', 'SSDSC', '850 PRO', '870 EVO', '860 PRO', '5300']
df['jeSSD'] = df['model'].apply(lambda x: 1 if any(k in str(x).upper() for k in ssd_keywords) else 0)

def getModel(model):
    m = str(model).upper()
    if m.startswith('ST') or 'SEAGATE' in m: return 'Seagate'
    if m.startswith('WDC') or m.startswith('WD') or 'WESTERN' in m: return 'Western Digital'
    if m.startswith('HGST') or m.startswith('HUH') or m.startswith('HMS'): return 'HGST'
    if m.startswith('TOSHIBA') or m.startswith('MG'): return 'Toshiba'
    if m.startswith('SAMSUNG'): return 'Samsung'
    if m.startswith('CT') or 'CRUCIAL' in m: return 'Crucial'
    return 'Other'

df['model'] = df['model'].apply(getModel)

display(df)
ostali = df[df['model'] == 'Other']['model'].unique()
print("nekategorizirani modeli: ", ostali)
modeli = df['model'].unique()
display("kategorizirani modeli: ", modeli)

vrste diskov:  ['CT250MX500SSD1' 'HGST HUH728080ALE604'
 'Seagate BarraCuda SSD ZA250CM10002' 'ST500LM012 HN'
 'HGST HUH728080ALE600' 'ST8000NM000A' 'TOSHIBA MQ01ABF050M' 'ST8000DM002'
 'ST8000NM0055' 'ST8000DM005' 'ST500LM030' 'ST16000NM001G' 'ST16000NM000J'
 'WD Blue SA510 2.5 250GB' 'WDC WUH721816ALE6L4' 'TOSHIBA MG08ACA16TA'
 'HGST HUS728T8TALE6L4' 'WDC WD5000LPCX' 'TOSHIBA HDWF180'
 'WDC WDS250G2B0A' 'Micron 5300 MTFDDAK480TDS' 'TOSHIBA MG08ACA16TE'
 'WUH721816ALE6L4' 'TOSHIBA MQ01ABF050'
 'Seagate BarraCuda 120 SSD ZA250CM10003' 'ST12000NM0008' 'ST16000NM002J'
 'ST12000NM000J' 'WDC WD5000LPVX' 'Seagate BarraCuda SSD ZA500CM10002'
 'ST10000NM001G' 'WDC WD5000BPKT' 'ST12000NM001G' 'ST10000NM0086'
 'ST12000NM003G' 'TOSHIBA MG07ACA14TA' 'HGST HUH721212ALE604'
 'WDC WUH721414ALE6L4' 'Seagate IronWolf ZA250NM10002'
 'TOSHIBA MG07ACA14TEY' 'Seagate FireCuda 120 SSD ZA500GM10001'
 'TOSHIBA MG09ACA16TE' 'Samsung SSD 850 PRO 1TB' 'Samsung SSD 870 EVO 2TB'
 'Samsung SSD 850 EVO 1TB' 'DELLBO

,model,capacity_bytes,failure,smart_1_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_9_raw,smart_10_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_192_raw,smart_193_raw,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,jeSSD
0,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,267.0,NaN,15.0,0.0,NaN,NaN,NaN,30.0,0.0,0.0,0.0,1
1,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,23328.0,NaN,3.0,0.0,NaN,NaN,NaN,35.0,0.0,0.0,0.0,1
2,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,9876.0,NaN,6.0,0.0,NaN,NaN,NaN,33.0,0.0,0.0,0.0,1
3,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,4994.0,NaN,2.0,0.0,NaN,NaN,NaN,36.0,0.0,0.0,0.0,1
4,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,1137.0,NaN,13.0,0.0,NaN,NaN,NaN,28.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329482,Western Digital,22000969973760,0,0.0,275.0,9.0,0.0,0.0,7691.0,0.0,9.0,NaN,NaN,328.0,328.0,32.0,0.0,0.0,0.0,0
329483,Western Digital,22000969973760,0,0.0,240.0,9.0,0.0,0.0,7691.0,0.0,9.0,NaN,NaN,327.0,327.0,26.0,0.0,0.0,0.0,0
329484,Western Digital,22000969973760,0,0.0,267.0,9.0,0.0,0.0,7686.0,0.0,9.0,NaN,NaN,327.0,327.0,32.0,0.0,0.0,0.0,0
329485,Western Digital,22000969973760,0,0.0,337.0,10.0,0.0,0.0,7700.0,0.0,10.0,NaN,NaN,330.0,330.0,29.0,0.0,0.0,0.0,0


nekategorizirani modeli:  ['Other']


'kategorizirani modeli: '

array(['Crucial', 'HGST', 'Seagate', 'Toshiba', 'Western Digital',
       'Other', 'Samsung'], dtype=object)

**Temeljit Preprocessing**

POMEMBNO - Zapolni NaN v teh nujnih stolpcih z 0
Ker XGBoost ne mara popolnih praznin, manjše število NaN pa zna rešiti, 

In [5]:
display(df)

NaNbefore = df.isnull().sum()
print("trenutno Nan: ", NaNbefore)

#pretvorimo byte v gigabyte
df = df.rename(columns={"capacity_bytes":"capacity_gigabytes"})
df['capacity_gigabytes'] = (df['capacity_gigabytes'] / (1024 ** 3)).round(2)
display(df)

#VSE NaN VREDNOSTI

#ŠTEVCI NAPAK (Error Counters)
#tukaj je "0" kul, ker predpostavimo, da če ni podatka, ni bilo napak.
#to velja tako za SSD kot za HDD.

stevcneNapake = ['smart_1_raw', 'smart_7_raw', 'smart_10_raw', 'smart_192_raw', 'smart_199_raw']
df[stevcneNapake] = df[stevcneNapake].fillna(0)

#MEHANSKI CIKLI (Mechanical Cycles) - PAZI: SSD vs HDD
#za SSD (jeSSD == 1) damo 0, ker nima motorja in glav.
#za HDD (jeSSD == 0) pa ne smemo dati 0, zato uporabimo mediano (srednjo vrednost).
mehanskiStolpci = ['smart_3_raw', 'smart_4_raw', 'smart_193_raw']

for col in mehanskiStolpci:
    # 1. Za SSD varno zapolnimo z 0
    df.loc[(df['jeSSD'] == 1) & (df[col].isnull()), col] = 0
    
    # 2. Za HDD izračunamo mediano (da ne uničimo podatkov z 0)
    mediana_hdd = df.loc[df['jeSSD'] == 0, col].median()
    df.loc[(df['jeSSD'] == 0) & (df[col].isnull()), col] = mediana_hdd

#SPLOŠNA PORABA (Power Cycle)
#vsak disk se vklopi, zato tukaj uporabimo povprečje, ker 0 ni realna.
df['smart_12_raw'] = df['smart_12_raw'].fillna(df['smart_12_raw'].median())

#TEMPERATURA, ITD..
nujniOstali = ['smart_5_raw', 'smart_9_raw', 'smart_187_raw', 'smart_188_raw', 'smart_194_raw', 'smart_197_raw', 'smart_198_raw']
df[nujniOstali] = df[nujniOstali].fillna(0)

NaNafter = df.isnull().sum()
print("NaN po procesiranju ", NaNafter)
print(df.head())

#df.to_csv('izbolsani_podatki_sample.csv', index=False)

,model,capacity_gigabytes,failure,smart_1_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_9_raw,smart_10_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_192_raw,smart_193_raw,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,jeSSD
0,Crucial,232.89,0,0.0,0.0,0.0,0.0,0.0,267.0,0.0,15.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,1
1,Crucial,232.89,0,0.0,0.0,0.0,0.0,0.0,23328.0,0.0,3.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,1
2,Crucial,232.89,0,0.0,0.0,0.0,0.0,0.0,9876.0,0.0,6.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,1
3,Crucial,232.89,0,0.0,0.0,0.0,0.0,0.0,4994.0,0.0,2.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1
4,Crucial,232.89,0,0.0,0.0,0.0,0.0,0.0,1137.0,0.0,13.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329482,Western Digital,20490.00,0,0.0,275.0,9.0,0.0,0.0,7691.0,0.0,9.0,0.0,0.0,328.0,328.0,32.0,0.0,0.0,0.0,0
329483,Western Digital,20490.00,0,0.0,240.0,9.0,0.0,0.0,7691.0,0.0,9.0,0.0,0.0,327.0,327.0,26.0,0.0,0.0,0.0,0
329484,Western Digital,20490.00,0,0.0,267.0,9.0,0.0,0.0,7686.0,0.0,9.0,0.0,0.0,327.0,327.0,32.0,0.0,0.0,0.0,0
329485,Western Digital,20490.00,0,0.0,337.0,10.0,0.0,0.0,7700.0,0.0,10.0,0.0,0.0,330.0,330.0,29.0,0.0,0.0,0.0,0


trenutno Nan:  model                 0
capacity_gigabytes    0
failure               0
smart_1_raw           0
smart_3_raw           0
smart_4_raw           0
smart_5_raw           0
smart_7_raw           0
smart_9_raw           0
smart_10_raw          0
smart_12_raw          0
smart_187_raw         0
smart_188_raw         0
smart_192_raw         0
smart_193_raw         0
smart_194_raw         0
smart_197_raw         0
smart_198_raw         0
smart_199_raw         0
jeSSD                 0
dtype: int64


,model,capacity_gigabytes,failure,smart_1_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_9_raw,smart_10_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_192_raw,smart_193_raw,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,jeSSD
0,Crucial,0.0,0,0.0,0.0,0.0,0.0,0.0,267.0,0.0,15.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,1
1,Crucial,0.0,0,0.0,0.0,0.0,0.0,0.0,23328.0,0.0,3.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,1
2,Crucial,0.0,0,0.0,0.0,0.0,0.0,0.0,9876.0,0.0,6.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,1
3,Crucial,0.0,0,0.0,0.0,0.0,0.0,0.0,4994.0,0.0,2.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1
4,Crucial,0.0,0,0.0,0.0,0.0,0.0,0.0,1137.0,0.0,13.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329482,Western Digital,0.0,0,0.0,275.0,9.0,0.0,0.0,7691.0,0.0,9.0,0.0,0.0,328.0,328.0,32.0,0.0,0.0,0.0,0
329483,Western Digital,0.0,0,0.0,240.0,9.0,0.0,0.0,7691.0,0.0,9.0,0.0,0.0,327.0,327.0,26.0,0.0,0.0,0.0,0
329484,Western Digital,0.0,0,0.0,267.0,9.0,0.0,0.0,7686.0,0.0,9.0,0.0,0.0,327.0,327.0,32.0,0.0,0.0,0.0,0
329485,Western Digital,0.0,0,0.0,337.0,10.0,0.0,0.0,7700.0,0.0,10.0,0.0,0.0,330.0,330.0,29.0,0.0,0.0,0.0,0


NaN po procesiranju  model                 0
capacity_gigabytes    0
failure               0
smart_1_raw           0
smart_3_raw           0
smart_4_raw           0
smart_5_raw           0
smart_7_raw           0
smart_9_raw           0
smart_10_raw          0
smart_12_raw          0
smart_187_raw         0
smart_188_raw         0
smart_192_raw         0
smart_193_raw         0
smart_194_raw         0
smart_197_raw         0
smart_198_raw         0
smart_199_raw         0
jeSSD                 0
dtype: int64
     model  capacity_gigabytes  failure  smart_1_raw  smart_3_raw  \
0  Crucial                 0.0        0          0.0          0.0   
1  Crucial                 0.0        0          0.0          0.0   
2  Crucial                 0.0        0          0.0          0.0   
3  Crucial                 0.0        0          0.0          0.0   
4  Crucial                 0.0        0          0.0          0.0   

   smart_4_raw  smart_5_raw  smart_7_raw  smart_9_raw  smart_10_raw  \


,model,capacity_gigabytes,failure,smart_1_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_9_raw,smart_10_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_192_raw,smart_193_raw,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,jeSSD
24793,HGST,0.0,1,0.0,403.0,20.0,0.0,0.0,41083.0,0.0,14.0,0.0,0.0,1792.0,1792.0,28.0,91.0,94.0,0.0,0
24823,HGST,0.0,1,0.0,401.0,18.0,76.0,0.0,41090.0,0.0,14.0,0.0,0.0,1835.0,1835.0,28.0,352.0,360.0,0.0,0
98004,HGST,0.0,1,0.0,52.0,279.0,0.0,0.0,57872.0,0.0,279.0,0.0,0.0,2813.0,2813.0,26.0,0.0,0.0,0.0,0
102179,HGST,0.0,1,3014678.0,404.0,30.0,16.0,0.0,55911.0,0.0,30.0,0.0,0.0,2108.0,2108.0,42.0,178.0,64.0,0.0,0
102249,HGST,0.0,1,4587589.0,407.0,18.0,507.0,0.0,55898.0,0.0,14.0,0.0,0.0,3284.0,3284.0,33.0,453.0,817.0,0.0,0
241207,Toshiba,0.0,1,0.0,8270.0,15.0,7.0,0.0,34152.0,0.0,15.0,0.0,0.0,14.0,247.0,45.0,0.0,0.0,0.0,0
288562,Western Digital,0.0,1,0.0,353.0,22.0,0.0,0.0,19917.0,0.0,22.0,0.0,0.0,1060.0,1060.0,37.0,0.0,0.0,0.0,0
311063,Seagate,0.0,1,86146472.0,0.0,21.0,0.0,532733945.0,5763.0,0.0,21.0,0.0,0.0,17.0,275.0,31.0,0.0,0.0,0.0,0


In [7]:
import gc
import pandas as pd

# Izbrišemo velike objekte, če obstajajo
if 'df' in locals():
    del df
if 'seznamDf' in locals():
    del seznamDf

# Ročno sprožimo "Garbage Collector", da sprosti RAM
gc.collect()

# Ponovno inicializiramo prazen seznam za nov krog
seznamDf = []